# Load dataset

In [ ]:
import csv

In [ ]:
dataset = []
with open('/root/data/lice_detection/lice_dataset_fish_only.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        dataset.append(row)

In [ ]:
len(dataset)

# Plot random image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import matplotlib.patches as patches

In [ ]:
randindex = np.random.randint(0, len(dataset))
randlice = dataset[randindex]
img = Image.open(randlice[0])
print(img.size)
rectangle = [int(coord) for coord in randlice[1:5]]
rec = [rectangle[0], rectangle[1], rectangle[2]-rectangle[0], rectangle[3]-rectangle[1]] 
ImageDraw.Draw(img).rectangle(rectangle, outline='white')

In [ ]:
img

In [ ]:
fig,ax = plt.subplots(1, figsize=(20, 10))
ax.imshow(np.array(img))

# Create a Rectangle patch
rect = patches.Rectangle((rec[0], rec[1]), rec[2], rec[3], linewidth=2, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

# Define augmentation

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa

In [ ]:
# load image
randindex = np.random.randint(0, len(dataset))
randlice = dataset[randindex]
rec = [int(coord) for coord in randlice[1:5]] 
img = Image.open(randlice[0])
image = np.array(img)

In [ ]:
# draw bboxes
bbs = ia.BoundingBoxesOnImage([ia.BoundingBox(x1=rec[0], y1=rec[1], x2=rec[2], y2=rec[3])], shape=image.shape)

In [ ]:
# ia.seed(1)
print(image.shape)
# image = ia.quokka(size=(256, 256))
# bbs = ia.BoundingBoxesOnImage([
#     ia.BoundingBox(x1=65, y1=100, x2=200, y2=150),
#     ia.BoundingBox(x1=150, y1=80, x2=200, y2=130)
# ], shape=image.shape)

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect BBs
    iaa.Affine(
        scale={"x": (0.7, 1.3), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.5, 0.5), "y": (-0.2, 0.2)},
        rotate=(-20, 20),
        shear=(-16, 16)
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])

# Make our sequence deterministic.
# We can now apply it to the image and then to the BBs and it will
# lead to the same augmentations.
# IMPORTANT: Call this once PER BATCH, otherwise you will always get the
# exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

# Augment BBs and images.
# As we only have one image and list of BBs, we use
# [image] and [bbs] to turn both into lists (batches) for the
# functions and then [0] to reverse that. In a real experiment, your
# variables would likely already be lists.
image_aug = seq_det.augment_images([image])[0]
bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

# print coordinates before/after augmentation (see below)
# use .x1_int, .y_int, ... to get integer coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
        i,
        before.x1, before.y1, before.x2, before.y2,
        after.x1, after.y1, after.x2, after.y2)
    )

# image with BBs before/after augmentation (shown below)
image_before = bbs.draw_on_image(image, thickness=3)
image_after = bbs_aug.draw_on_image(image_aug, thickness=3, color=[0, 255, 255])

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(image_before)

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(image_after)

# NOW USE RETINANET

In [ ]:
import keras_retinanet
from keras_retinanet.bin.train import * 
import os

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
backbone = models.backbone('vgg16')

In [ ]:
generator_parameters = {
    'batch_size'       : 1,
    'image_min_side'   : 3000,
    'image_max_side'   : 4000
}

In [ ]:
def create_generators(args, preprocess_image):
    """ Create generators for training and validation.
    Args
        args             : parseargs object containing configuration for generators.
        preprocess_image : Function that preprocesses an image for the network.
    """
    common_args = {
        'batch_size'       : 1,
        'image_min_side'   : 4000,
        'image_max_side'   : 4000,
        'preprocess_image' : preprocess_image,
    }
    args.dataset_type == 'csv':
    train_generator = CSVGenerator(
        args.annotations,
        args.classes,
        transform_generator=transform_generator,
        **common_args
    )

    if args.val_annotations:
        validation_generator = CSVGenerator(
            args.val_annotations,
            args.classes
        )
    else:
        validation_generator = None
    return train_generator, validation_generator